In [1]:
import pandas as pd
import numpy as np

from xgboost import XGBClassifier
from feature_engine.selection import DropConstantFeatures
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import mutual_info_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

#### Reading the data

In [2]:
fraud_df = pd.read_csv("fraud_detection.csv")


#### Basic EDA

In [3]:
# fraud_df.head()
# fraud_df.shape
fraud_df.columns

Index(['Transaction ID', 'User ID', 'Transaction Amount',
       'Transaction Date and Time', 'Merchant ID', 'Payment Method',
       'Country Code', 'Transaction Type', 'Device Type', 'IP Address',
       'Browser Type', 'Operating System', 'Merchant Category', 'User Age',
       'User Occupation', 'User Income', 'User Gender', 'User Account Status',
       'Transaction Status', 'Location Distance', 'Time Taken for Transaction',
       'Transaction Time of Day', 'User's Transaction History',
       'Merchant's Reputation Score', 'User's Device Location',
       'Transaction Currency', 'Transaction Purpose', 'User's Credit Score',
       'User's Email Domain', 'Merchant's Business Age',
       'Transaction Authentication Method', 'Fraudulent Flag'],
      dtype='object')

In [4]:
features = ["transaction_amount", "payment_method", "country_code", "transaction_type",
 "device_type", "ip_address", "browser_type", "operating_system", "merchant_category",
            "user_age", "user_occupation", "user_income", "user_gender", 'user_account_status',
            'transaction_status', 'location_distance', 'time_taken_for_transaction',
            'transaction_time_of_day', 'user\'s_transaction_history',
            'merchant\'s_reputation_score', 'user\'s_device_location',
            'transaction_currency', 'transaction_purpose', 'user\'s_credit_score',
            'user\'s_email_domain', 'merchant\'s_business_age',
            'transaction_authentication_method']


In [5]:
# formatting the column names to be more consistent
fraud_df.columns = fraud_df.columns.str.lower()
fraud_df.columns = fraud_df.columns.str.replace(" ", "_")

### Feature Engineering

In [6]:
# for feature in features:
#     score = mutual_info_score(fraud_df[feature], y)
#     print(f"{feature} score: {score}")


In [17]:
selected_features = ["user_income", "transaction_amount",
                     "location_distance", "time_taken_for_transaction",
                     "user's_credit_score", "merchant_category", "merchant's_reputation_score",
                     "transaction_time_of_day", "transaction_authentication_method", 'user\'s_transaction_history',
                     'user_account_status', 'user_age', 'transaction_currency']

display(fraud_df[selected_features].head())

# seperating the data into target (y) and predictor (x) variables.
y = fraud_df["fraudulent_flag"]
X = fraud_df[selected_features]

,user_income,transaction_amount,location_distance,time_taken_for_transaction,user's_credit_score,merchant_category,merchant's_reputation_score,transaction_time_of_day,transaction_authentication_method,user's_transaction_history,user_account_status,user_age,transaction_currency
0,66826.21,163.08,9.34,24.22,343,Industrial & Scientific,2.71,Daytime,Bluetooth Authentication,26,Pro,68,NOK
1,89356.71,430.74,65.28,55.11,688,Beauty & Cosmetics,3.95,Daytime,NFC Tag,60,Pending Approval,22,EGP
2,58438.63,415.74,44.05,53.84,371,Real Estate,3.81,Daytime,Token,81,Pro,71,MXN
3,3426.92,565.89,21.70,21.62,687,Appliances,2.67,Daytime,Time-Based OTP,18,Premium,78,CLP
4,53080.12,955.49,56.63,53.71,605,Jewelry,3.19,Daytime,Password,98,Free,31,RUB


### Encoding categorical variables

In [18]:
encoder = LabelEncoder()
merchant = encoder.fit_transform(fraud_df["merchant_category"])
time = encoder.fit_transform(fraud_df["transaction_time_of_day"])
auth = encoder.fit_transform(fraud_df["transaction_authentication_method"])
account_status = encoder.fit_transform(fraud_df["user_account_status"])
currency = encoder.fit_transform(fraud_df["transaction_currency"])

In [19]:
merchant = pd.DataFrame(merchant)
time = pd.DataFrame(time)
auth = pd.DataFrame(auth)
account_status = pd.DataFrame(account_status)
currency = pd.DataFrame(currency)

cat_col = pd.concat([merchant, time, auth, account_status, currency], axis=1)
cat_col= pd.concat([cat_col.iloc[:, 0].rename("merchant", inplace=True), 
           cat_col.iloc[:, 1].rename("time", inplace=True),
           cat_col.iloc[:, 2].rename("auth", inplace=True),
           cat_col.iloc[:, 3].rename("account_status", inplace=True),
           cat_col.iloc[:, 4].rename("currency", inplace=True)], axis=1)



### Feature scaling

In [20]:
# dropping the categorical variables
print(X.dtypes)

X = X.drop(["merchant_category",
        "transaction_time_of_day",
        "transaction_authentication_method",
        'user\'s_transaction_history',
        'user_account_status',
        'transaction_currency'], axis=1)

user_income                          float64
transaction_amount                   float64
location_distance                    float64
time_taken_for_transaction           float64
user's_credit_score                    int64
merchant_category                     object
merchant's_reputation_score          float64
transaction_time_of_day               object
transaction_authentication_method     object
user's_transaction_history             int64
user_account_status                   object
user_age                               int64
transaction_currency                  object
dtype: object


In [21]:
# adding the encoded variables
X = pd.concat([X, cat_col], axis=1)
X.head()

,user_income,transaction_amount,location_distance,time_taken_for_transaction,user's_credit_score,merchant's_reputation_score,user_age,merchant,time,auth,account_status,currency
0,66826.21,163.08,9.34,24.22,343,2.71,68,22,0,5,11,24
1,89356.71,430.74,65.28,55.11,688,3.95,22,4,0,18,9,11
2,58438.63,415.74,44.05,53.84,371,3.81,71,29,0,33,11,22
3,3426.92,565.89,21.70,21.62,687,2.67,78,0,0,32,10,6
4,53080.12,955.49,56.63,53.71,605,3.19,31,23,0,21,3,31


In [22]:
# using standard scaler

scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(X)

In [29]:
scaled_df

array([[0.66491114, 0.16224224, 0.08424242, ..., 0.13157895, 0.64705882,
        0.61538462],
       [0.892492  , 0.43017017, 0.64929293, ..., 0.47368421, 0.52941176,
        0.28205128],
       [0.5801881 , 0.41515516, 0.43484848, ..., 0.86842105, 0.64705882,
        0.56410256],
       ...,
       [0.32608421, 0.19647648, 0.91090909, ..., 0.42105263, 0.52941176,
        0.35897436],
       [0.05955668, 0.35768769, 0.9020202 , ..., 0.        , 0.88235294,
        0.        ],
       [0.397202  , 0.65474474, 0.53454545, ..., 0.55263158, 0.82352941,
        0.61538462]])

#### Splitting the dataset into train and test

In [30]:
X_train, X_test, y_train, y_test = train_test_split(scaled_df, y, test_size=0.3)

In [31]:
# X_train.reset_index(drop=True, inplace=True)

AttributeError: 'numpy.ndarray' object has no attribute 'reset_index'

In [33]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
# X_train.head()

(4200000, 12)
(1800000, 12)
(4200000,)
(1800000,)


### Model selection and fitting

In [ ]:
# creating a parameter grid for Gridsearch

params = {
    "n_estimators": [1, 2, 3, 4, 5],
    "max_depth": [1, 2, 3, 4, 5],
    "learning_rate": [0.5, 1, 1.5, 2]
}

In [ ]:
# using GridSearchCV

grid_search = GridSearchCV(estimator=XGmodel, param_grid=params, cv=5,
                           verbose=)

In [34]:
XGmodel = XGBClassifier(n_estimators=3, max_depth=3,
                        learning_rate=1, objective='binary:logistic')

XGmodel.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=3, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [35]:
y_pred = XGmodel.predict(X_test)

accuracy_score(y_pred, y_test)

0.5003211111111111

In [36]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.50      0.42      0.46    899729
           1       0.50      0.58      0.54    900271

    accuracy                           0.50   1800000
   macro avg       0.50      0.50      0.50   1800000
weighted avg       0.50      0.50      0.50   1800000

